In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
import time
import random

In [3]:
def fetchUrl(pid, uid, max_id):
    
    url = "https://weibo.com/ajax/statuses/buildComments"
    
#     headers = {
#         'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.54 Safari/537.36'
 
# #         "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36",
#     }
    headers = {'User-Agent': random.choice(user_agent)
        , 'Cookie': random.choice(cookies)
               # 'Referer': 'https://m.weibo.cn/detail/4497103885505673',
               # 'Sec-Fetch-Mode': 'navigate'
    }  # 设置user-agent来进行伪装，突破微博反爬限制
    
    params = {
 
        "flow" : 1,# flow=0按热度，flow=1,按时间
        "is_reload" : 1,
        "id" : pid,
        "is_show_bulletin" : 2,
        "is_mix" : 0,
        "max_id" : max_id,
        "count" : 20,
        "uid" : uid,
    }
    r = requests.get(url, headers = headers, params = params,timeout=5)
#     print(r)
    return r

In [4]:
def parseJson(jsonObj):

    data = jsonObj["data"]
    max_id = jsonObj["max_id"]
    global i
    commentData = []
    for item in data:
        # 评论id
        comment_Id = item["id"]
        # 评论内容
        content = BeautifulSoup(item["text"], "html.parser").text
        # 评论时间
        created_at = item["created_at"]
        # 点赞数
        like_counts = item["like_counts"]
        # 评论数
        total_number = item["total_number"]
        
        # 评论者 id，name，city
        user = item["user"]
        userID = user["id"]
        userName = user["name"]
        userCity = user["location"]
#         print(i,' ',comment_Id)
        i+=1
        if i %100==0:
            print(i,time.time() - time_start)
            if i%500==0:
                time.sleep(5)
        dataItem = [comment_Id, created_at, userID, userName, userCity, like_counts, total_number, content]
        commentData.append(dataItem)
        
    return commentData, max_id

In [5]:
def save_data(data, path, filename):
    
    if not os.path.exists(path):
        os.makedirs(path)

    dataframe = pd.DataFrame(data)
    dataframe.to_csv(path + filename, encoding='utf_8_sig', mode='a', index=False, sep=',', header=False )

In [7]:
user_agent = [
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.54 Safari/537.36',
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/96.0.4664.110 Safari/537.36",
    'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.54 Safari/537.36',
#     'Mozilla/5.0 (iPhone; CPU iPhone OS 13_2_3 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.0.3 Mobile/15E148 Safari/604.1',
    
]
#         "Mozilla/5.0 (Macintosh; U; Intel Mac OS X 10_6_8; en-us) AppleWebKit/534.50 (KHTML, like Gecko) Version/5.1 Safari/534.50",
#         "Mozilla/5.0 (Windows; U; Windows NT 6.1; en-us) AppleWebKit/534.50 (KHTML, like Gecko) Version/5.1 Safari/534.50",
#         "Mozilla/5.0 (Windows NT 10.0; WOW64; rv:38.0) Gecko/20100101 Firefox/38.0",
#         "Mozilla/5.0 (Windows NT 10.0; WOW64; Trident/7.0; .NET4.0C; .NET4.0E; .NET CLR 2.0.50727; .NET CLR 3.0.30729; .NET CLR 3.5.30729; InfoPath.3; rv:11.0) like Gecko",
#         "Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Trident/5.0)",
#         "Mozilla/4.0 (compatible; MSIE 8.0; Windows NT 6.0; Trident/4.0)",
#         "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.0)",
#         "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1)",
#         "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.6; rv:2.0.1) Gecko/20100101 Firefox/4.0.1",
#         "Mozilla/5.0 (Windows NT 6.1; rv:2.0.1) Gecko/20100101 Firefox/4.0.1",
#         "Opera/9.80 (Macintosh; Intel Mac OS X 10.6.8; U; en) Presto/2.8.131 Version/11.11",
#         "Opera/9.80 (Windows NT 6.1; U; en) Presto/2.8.131 Version/11.11",
#         "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_7_0) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.56 Safari/535.11",
#         "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; Maxthon 2.0)",
#         "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; TencentTraveler 4.0)",
#         "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)",
#         "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; The World)",
#         "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; Trident/4.0; SE 2.X MetaSr 1.0; SE 2.X MetaSr 1.0; .NET CLR 2.0.50727; SE 2.X MetaSr 1.0)",
#         "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; 360SE)",
#         "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; Avant Browser)",
#         "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1)",
#         "Mozilla/5.0 (iPhone; U; CPU iPhone OS 4_3_3 like Mac OS X; en-us) AppleWebKit/533.17.9 (KHTML, like Gecko) Version/5.0.2 Mobile/8J2 Safari/6533.18.5",
#         "Mozilla/5.0 (iPod; U; CPU iPhone OS 4_3_3 like Mac OS X; en-us) AppleWebKit/533.17.9 (KHTML, like Gecko) Version/5.0.2 Mobile/8J2 Safari/6533.18.5",
#         "Mozilla/5.0 (iPad; U; CPU OS 4_3_3 like Mac OS X; en-us) AppleWebKit/533.17.9 (KHTML, like Gecko) Version/5.0.2 Mobile/8J2 Safari/6533.18.5",
#         "Mozilla/5.0 (Linux; U; Android 2.3.7; en-us; Nexus One Build/FRF91) AppleWebKit/533.1 (KHTML, like Gecko) Version/4.0 Mobile Safari/533.1",
#         "MQQBrowser/26 Mozilla/5.0 (Linux; U; Android 2.3.7; zh-cn; MB200 Build/GRJ22; CyanogenMod-7) AppleWebKit/533.1 (KHTML, like Gecko) Version/4.0 Mobile Safari/533.1",
#         "Opera/9.80 (Android 2.3.4; Linux; Opera Mobi/build-1107180945; U; en-GB) Presto/2.8.149 Version/11.10",
#         "Mozilla/5.0 (Linux; U; Android 3.0; en-us; Xoom Build/HRI39) AppleWebKit/534.13 (KHTML, like Gecko) Version/4.0 Safari/534.13",
#         "Mozilla/5.0 (BlackBerry; U; BlackBerry 9800; en) AppleWebKit/534.1+ (KHTML, like Gecko) Version/6.0.0.337 Mobile Safari/534.1+",
#         "Mozilla/5.0 (hp-tablet; Linux; hpwOS/3.0.0; U; en-US) AppleWebKit/534.6 (KHTML, like Gecko) wOSBrowser/233.70 Safari/534.6 TouchPad/1.0",
#         "Mozilla/5.0 (SymbianOS/9.4; Series60/5.0 NokiaN97-1/20.0.019; Profile/MIDP-2.1 Configuration/CLDC-1.1) AppleWebKit/525 (KHTML, like Gecko) BrowserNG/7.1.18124",
#         "Mozilla/5.0 (compatible; MSIE 9.0; Windows Phone OS 7.5; Trident/5.0; IEMobile/9.0; HTC; Titan)",
#         "UCWEB7.0.2.37/28/999",
#         "NOKIA5700/ UCWEB7.0.2.37/28/999",
#         "Openwave/ UCWEB7.0.2.37/28/999",
#         "Mozilla/4.0 (compatible; MSIE 6.0; ) Opera/UCWEB7.0.2.37/28/999",
#         # iPhone 6：
#         "Mozilla/6.0 (iPhone; CPU iPhone OS 8_0 like Mac OS X) AppleWebKit/536.26 (KHTML, like Gecko) Version/8.0 Mobile/10A5376e Safari/8536.25",
#     ]
cookies = [
    'SINAGLOBAL=6726116035866.517.1631512297243; UM_distinctid=17fc525dd9bdcf-08a0267fd550dc-9771a39-144000-17fc525dd9cbfe; UOR=www.lsdhss.com,s.weibo.com,www.lsdhss.com; SUBP=0033WrSXqPxfM725Ws9jqgMF55529P9D9WWizGvTng7WCpcJRD6k5L5A5JpX5KMhUgL.Fo-0ehMXeo5RSKz2dJLoIp7LxKML1KBLBKnLxKqL1hnLBoMfe05NShz71h-E; ALF=1683337271; SSOLoginState=1651801271; SCF=Al-y5H8D96wGBNLnt00kBhIjsdkDxuQvhbN4RVEnB2Sn-6lYj4PaVIiftG0GWIV46ElVHC8EuYITHeSjV3aKZog.; SUB=_2A25PcAznDeRhGeNN61UV8i7Ezj6IHXVsBHkvrDV8PUNbmtAKLXLVkW9NSclYPl2BjBvo4x6ifJ8XJr6oLTWDyViR; XSRF-TOKEN=hbjmVexOdOy2qVtNf3ViCx1X; _s_tentry=weibo.com; Apache=3572742405069.753.1651801305620; ULV=1651801305637:5:1:1:3572742405069.753.1651801305620:1648634367876; WBPSESS=oHMXVrt4Yl_vixynRB4TRXrdK6WvoTwwdq8A_sCNoPRBe6HBe3KaWmkHnvpirkayjppVK8wt828uQFbRI5h0drJ3X8d1IZs6r5Xjspt8VJo24AA5rPZMwwc46rgrJS7gRO3CFmxPbCVRMmMzfR7qtg==',
    'SINAGLOBAL=6726116035866.517.1631512297243; UM_distinctid=17fc525dd9bdcf-08a0267fd550dc-9771a39-144000-17fc525dd9cbfe; UOR=www.lsdhss.com,s.weibo.com,www.lsdhss.com; SUBP=0033WrSXqPxfM725Ws9jqgMF55529P9D9WWizGvTng7WCpcJRD6k5L5A5JpX5KMhUgL.Fo-0ehMXeo5RSKz2dJLoIp7LxKML1KBLBKnLxKqL1hnLBoMfe05NShz71h-E; ALF=1683337271; SSOLoginState=1651801271; SCF=Al-y5H8D96wGBNLnt00kBhIjsdkDxuQvhbN4RVEnB2Sn-6lYj4PaVIiftG0GWIV46ElVHC8EuYITHeSjV3aKZog.; SUB=_2A25PcAznDeRhGeNN61UV8i7Ezj6IHXVsBHkvrDV8PUNbmtAKLXLVkW9NSclYPl2BjBvo4x6ifJ8XJr6oLTWDyViR; XSRF-TOKEN=hbjmVexOdOy2qVtNf3ViCx1X; _s_tentry=weibo.com; Apache=3572742405069.753.1651801305620; ULV=1651801305637:5:1:1:3572742405069.753.1651801305620:1648634367876; WBPSESS=oHMXVrt4Yl_vixynRB4TRXrdK6WvoTwwdq8A_sCNoPRBe6HBe3KaWmkHnvpirkayxm03J7a3Z91b86oWdtc9HIiXkZRA8yTHbb_qCDl3KQ2pTU_80TEf2vBCz9iTlzWlUY-lthS-99VTMKTyhMAdNw=='# 微博的cookie
    ]

In [ ]:
if __name__ == "__main__":
    time_start = time.time()
    pids = [4597560524413356,4710420180899011,4752062316807824]  # 微博id，固定 4576013109691468,
    uid = 7524392184            # 用户id，固定
    path = "Data/"           # 保存的路径
    filename = "comments.csv"   # 保存的文件名
    csvHeader = [["评论id", "发布时间", "用户id", "用户昵称", "用户城市", "点赞数", "回复数", "评论内容"]]
    save_data(csvHeader, path, filename)
    for pid in pids:
        max_id = 0
        i = 0
        while True:
            html = fetchUrl(pid, uid, max_id)
            if html.status_code != 200:
                print('连接错误，返回',html.status_code)
                time.sleep(5)
                continue
            elif html.status_code ==200:
                 html=html.json()
            comments, max_id = parseJson(html)
            print('max_id',max_id)
            save_data(comments, path, filename)
            # max_id 为 0 时，表示爬取结束
            if max_id == 0:
                break

max_id 4675183711093581
max_id 4661529892360463
max_id 4660711185189755
max_id 4658973217135250
max_id 4656317345825771
100 2.446455478668213
max_id 4655012875076699
max_id 4654658112460319
max_id 4654524176795884
max_id 4649865793506522
max_id 4639801657265672
max_id 4639783160910850
max_id 4639782212209079
200 5.158259868621826
max_id 4639781192992170
max_id 4639780254255558
max_id 4639779041312819
max_id 4639777984614295
max_id 4639775497393493
max_id 4639766718454232
300 7.49840235710144
max_id 4639760989032577
max_id 4639745463027525
max_id 4639740337587805
max_id 4639738168871927
max_id 4639736347495465
max_id 4639734834659965
400 9.744679927825928
max_id 4639733152221186
max_id 4639729637130819
max_id 4639727310080812
max_id 4639720711655878
max_id 4639717179524328
500 11.834561347961426
max_id 4639714676838627
max_id 4639708174353780
max_id 4639705860409722
max_id 4639703402548027
max_id 4639687027725335
max_id 4639680001214867
600 19.203330039978027
max_id 4639677552528989
max

/home/ma-user/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:219: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)


1600 52.54425501823425
max_id 4598855934413270
max_id 4598806366914686
max_id 4598749583641869
max_id 4598690977166057
max_id 4598641342030251
max_id 4598497900497709
max_id 4598482490629932
1700 55.36238884925842
max_id 4598446273075775
max_id 4598422604351915
max_id 4598405349000663
max_id 4598379949065603
max_id 4598360566399923
max_id 4598318938727985
1800 58.066730260849
max_id 4598290233170787
max_id 4598279511487204
max_id 4598268094591355
max_id 4598258523181931
max_id 4598232498047788
max_id 4598159328151035
1900 61.45295524597168
max_id 4598133034329495
max_id 4598126914578851
max_id 4598121654668407
max_id 4598118659918764
max_id 4598115053341136
max_id 4598113618630592
2000 64.45925831794739
max_id 4598107831283976
max_id 4598101225253020
max_id 4598093301167986
max_id 4598085672767705
max_id 4598074196896943
max_id 4598063458697181
max_id 4598053522905911
2100 72.86413788795471
max_id 4598044568068060
max_id 4598036989485631
max_id 4598026817766451
max_id 4598021088614046


/home/ma-user/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:219: UserWarning: "b'..'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)


max_id 4597914762222567
2400 82.26049256324768
max_id 4597907427170321
max_id 4597903618736914
max_id 4597899001074800
max_id 4597895712475388
max_id 4597891329692073
max_id 4597883166791044
2500 85.2003870010376
max_id 4597875210192831
max_id 4597861163467111
max_id 4597827915481904
max_id 4597803391910424
max_id 4597796920099822
max_id 4597789710090267
2600 92.84225916862488
max_id 4597783954462945
max_id 4597779018026661
max_id 4597776862416158
max_id 4597772919514051
max_id 4597770960772951
max_id 4597769320800857
2700 96.2208423614502
max_id 4597766640374904
max_id 4597764673512875
max_id 4597762631401673
max_id 4597759573501763
max_id 4597756713511052
max_id 4597754532206207
2800 99.15397810935974
max_id 4597750895488831
max_id 4597749666809906
max_id 4597748219777590
max_id 4597744986228671
max_id 4597742657877881
2900 101.49682927131653
max_id 4597740623370064
max_id 4597739469686524
max_id 4597737591676576
max_id 4597734374388090
max_id 4597732507389883
max_id 4597730653512695